In [1]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt
train_on_gpu = torch.cuda.is_available()

In [2]:
def TrainConstructor(path_to_train, path_to_valid, num_workers=4):

    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize([299,299]),
        transforms.ToTensor()])

    train_dataset = datasets.ImageFolder(path_to_train, transform=train_transform)
    print("Successfully Loaded Training Set.")
    valid_dataset = datasets.ImageFolder(path_to_valid, transform=train_transform)
    print("Successfully Loaded Validation Set.")

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, num_workers=num_workers, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=2, num_workers=num_workers, shuffle=True)

    train_category = path_to_train.split('/')[-1]

    return train_loader, valid_loader, train_category

In [ ]:
train_loader, valid_loader, train_category = TrainConstructor(
path_to_train='C:/Users/fuzha/Downloads/progan_train/car',
path_to_valid='C:/Users/fuzha/Downloads/progan_val/car', num_workers=1)

## Model



In [2]:

class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x


class Block(nn.Module):
    def __init__(self,in_filters,out_filters,reps,strides=1,start_with_relu=True,grow_first=True):
        super(Block, self).__init__()

        if out_filters != in_filters or strides!=1:
            self.skip = nn.Conv2d(in_filters,out_filters,1,stride=strides, bias=False)
            self.skipbn = nn.BatchNorm2d(out_filters)
        else:
            self.skip=None

        rep=[]

        filters=in_filters
        if grow_first:
            rep.append(nn.ReLU(inplace=True))
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))
            filters = out_filters

        for i in range(reps-1):
            rep.append(nn.ReLU(inplace=True))
            rep.append(SeparableConv2d(filters,filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(filters))

        if not grow_first:
            rep.append(nn.ReLU(inplace=True))
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))

        if not start_with_relu:
            rep = rep[1:]
        else:
            rep[0] = nn.ReLU(inplace=False)

        if strides != 1:
            rep.append(nn.MaxPool2d(3,strides,1))
        self.rep = nn.Sequential(*rep)

    def forward(self,inp):
        x = self.rep(inp)

        if self.skip is not None:
            skip = self.skip(inp)
            skip = self.skipbn(skip)
        else:
            skip = inp

        x+=skip
        return x


class Xception(nn.Module):
    def __init__(self, num_classes=1000):
        super(Xception, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(3, 32, 3,2, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(32,64,3,bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        #do relu here

        self.block1=Block(64,128,2,2,start_with_relu=False,grow_first=True)
        self.block2=Block(128,256,2,2,start_with_relu=True,grow_first=True)
        self.block3=Block(256,728,2,2,start_with_relu=True,grow_first=True)

        self.block4=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block5=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block6=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block7=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block8=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block9=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block10=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block11=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block12=Block(728,1024,2,2,start_with_relu=True,grow_first=False)

        self.conv3 = SeparableConv2d(1024,1536,3,1,1)
        self.bn3 = nn.BatchNorm2d(1536)
        self.relu3 = nn.ReLU(inplace=True)

        #do relu here
        self.conv4 = SeparableConv2d(1536,2048,3,1,1)
        self.bn4 = nn.BatchNorm2d(2048)

        self.fc = nn.Linear(2048, num_classes)

        # #------- init weights --------
        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm2d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()
        # #-----------------------------

    def features(self, input):
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)
        x = self.block8(x)
        x = self.block9(x)
        x = self.block10(x)
        x = self.block11(x)
        x = self.block12(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv4(x)
        x = self.bn4(x)
        return x

    def logits(self, features):
        x = nn.ReLU(inplace=True)(features)

        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, input):
        x = self.features(input)
        x = self.logits(x)
        return x


def xception(num_classes=2):
    model = Xception(num_classes=num_classes)
    model.last_linear = model.fc
    del model.fc
    return model.cuda()

In [3]:
model = xception(num_classes=2)
model = nn.DataParallel(model)

## Loss Function and Optimizer

In [4]:
# # specify loss function (categorical cross-entropy)
# criterion = nn.CrossEntropyLoss()

# # specify optimizer
# optimizer = optimizer = torch.optim.SGD([
#     {'params': list(model.parameters())[:-1], 'lr': 1e-3, 'momentum': 0.9, 'weight_decay': 1e-3},
#     {'params': list(model.parameters())[-1], 'lr': 5e-5, 'momentum': 0.9, 'weight_decay': 1e-5}
# ])

## Train


In [7]:
# train_categories = ['car','cat','horse','person','sofa']

# for j in range(4):
# print('Training on category ' + 'sofa' + ':')
# train_loader, valid_loader = TrainConstructor(path_to_train='E:/Downloads/progan_train/sofa',path_to_valid='E:/Downloads/progan_val/sofa', num_workers=24)

# model = xception(num_classes=2)
# model = nn.DataParallel(model)

# # specify loss function (categorical cross-entropy)
# criterion = nn.CrossEntropyLoss()

# # specify optimizer
# optimizer = optimizer = torch.optim.SGD([
#     {'params': list(model.parameters())[:-1], 'lr': 1e-3, 'momentum': 0.9, 'weight_decay': 1e-3},
#     {'params': list(model.parameters())[-1], 'lr': 5e-5, 'momentum': 0.9, 'weight_decay': 1e-5}
# ])

# number of epochs to train the model
n_epochs = 15

valid_loss_min = np.Inf # track change in validation loss
training_vis = []
valid_vis = []

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

    torch.cuda.empty_cache()    
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)

    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)

    training_vis.append(train_loss)
    valid_vis.append(valid_loss)

    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
#     torch.save(model.state_dict(), 'Xception_' + 'sofa' + '_' + str(epoch) +'.pt')

    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...\n'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'Xception_' + 'car' + '_best.pt')
        valid_loss_min = valid_loss

np.savetxt('Xception_' + 'car' + '.txt', np.array([training_vis, valid_vis]))

plt.plot(range(epoch), training_vis)
plt.scatter(range(epoch), training_vis)
plt.scatter(range(epoch), valid_vis)
plt.plot(range(epoch), valid_vis)
plt.savefig('Xception_' + 'car' + '.svg')
plt.clf()

## Load

In [3]:
model = xception(num_classes=2)
model = nn.DataParallel(model)
criterion = nn.CrossEntropyLoss()
model.load_state_dict(torch.load('Xception_car_best.pt'))

<All keys matched successfully>

## Test

In [4]:
def TestConstructor(path_to_test, classes, num_workers=4):
    
    # Transformations to the image, edit as need be
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize([299,299]),
        transforms.ToTensor()])

    test_dataset = datasets.ImageFolder(path_to_test, transform=transform)
    print("Successfully Loaded Test Set.")
    
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, 
        num_workers=num_workers)
    if classes != None:
        print("Number of Classes:", len(classes))
    return test_loader, classes

In [ ]:
test_categories = ['airplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

for j in range(20):
    print('For test category ' + test_categories[j] + ':')
    test_loader, classes = TestConstructor(path_to_test='C:/Users/fuzha/Downloads/CNN_synth_testset/progan/' + test_categories[j], classes=['Real','Fake'],num_workers=1)

# test_loader, classes = TestConstructor(path_to_test='C:/Users/fuzha/Downloads/CNN_synth_testset/progan/car', classes=['Real','Fake'], num_workers=1)

# track test loss
    test_loss = 0.0
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))

    model.eval()
    # iterate over test data
    for data, target in test_loader:
    #     print(target)
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
    #     print(output)
        # calculate the batch loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
    #     print(pred)
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(2):
    #         print(i)
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # average test loss
    test_loss = test_loss/len(test_loader.dataset)
    print('Test Loss: {:.6f}'.format(test_loss))

    for i in range(2):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                classes[i], 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    print('Test Accuracy (Overall): %2d%% (%2d/%2d)\n' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))

In [6]:
train_categories = ['car', 'cat', 'horse', 'person', 'sofa']
test_gans = ['biggan','cyclegan','deepfake','san','stargan','stylegan','stylegan2','whichfaceisreal']
cyclegan_categories = ['apple','horse','orange','summer','winter','zebra']
stylegan_categories = ['bedroom','car','cat']
stylegan2_categories = ['car','cat','church','horse']

for train_category in train_categories:
    print('For train category ' + train_category + ':')
    model = xception(num_classes=2)
    model = nn.DataParallel(model)
    criterion = nn.CrossEntropyLoss()
    model.load_state_dict(torch.load('Xception_' + train_category + '_best.pt'))

    for test_gan in test_gans:
        print('For test gan ' + test_gan + ':')
        test_categories = []
        if test_gan == 'cyclegan':
            test_categories = cyclegan_categories
        elif test_gan == 'stylegan':
            test_categories = stylegan_categories
        elif test_gan == 'stylegan2':
            test_categories = stylegan2_categories

        if len(test_categories) == 0:
            test_loader, classes = TestConstructor(path_to_test='C:/Users/fuzha/Downloads/CNN_synth_testset/' + test_gan, classes=['Real','Fake'], num_workers=1)

            # track test loss
            test_loss = 0.0
            class_correct = list(0. for i in range(2))
            class_total = list(0. for i in range(2))

            model.eval()
            # iterate over test data
            for data, target in test_loader:
            #     print(target)
                # move tensors to GPU if CUDA is available
                if train_on_gpu:
                    data, target = data.cuda(), target.cuda()
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model(data)
            #     print(output)
                # calculate the batch loss
                loss = criterion(output, target)
                # update test loss 
                test_loss += loss.item()*data.size(0)
                # convert output probabilities to predicted class
                _, pred = torch.max(output, 1)
            #     print(pred)
                # compare predictions to true label
                correct_tensor = pred.eq(target.data.view_as(pred))
                correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
                # calculate test accuracy for each object class
                for i in range(2):
            #         print(i)
                    label = target.data[i]
                    class_correct[label] += correct[i].item()
                    class_total[label] += 1

            # average test loss
            test_loss = test_loss/len(test_loader.dataset)
            print('Test Loss: {:.6f}'.format(test_loss))

            for i in range(2):
                if class_total[i] > 0:
                    print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                        classes[i], 100 * class_correct[i] / class_total[i],
                        np.sum(class_correct[i]), np.sum(class_total[i])))
                else:
                    print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

            print('Test Accuracy (Overall): %2d%% (%2d/%2d)\n' % (
                100. * np.sum(class_correct) / np.sum(class_total),
                np.sum(class_correct), np.sum(class_total)))

        else:
            for test_category in test_categories:
                print('For test category ' + test_category + ':')
                test_loader, classes = TestConstructor(path_to_test='C:/Users/fuzha/Downloads/CNN_synth_testset/' + test_gan + '/' + test_category, classes=['Real','Fake'], num_workers=1)

                # track test loss
                test_loss = 0.0
                class_correct = list(0. for i in range(2))
                class_total = list(0. for i in range(2))

                model.eval()
                # iterate over test data
                for data, target in test_loader:
                #     print(target)
                    # move tensors to GPU if CUDA is available
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()
                    # forward pass: compute predicted outputs by passing inputs to the model
                    output = model(data)
                #     print(output)
                    # calculate the batch loss
                    loss = criterion(output, target)
                    # update test loss 
                    test_loss += loss.item()*data.size(0)
                    # convert output probabilities to predicted class
                    _, pred = torch.max(output, 1)
                #     print(pred)
                    # compare predictions to true label
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
                    # calculate test accuracy for each object class
                    for i in range(2):
                #         print(i)
                        label = target.data[i]
                        class_correct[label] += correct[i].item()
                        class_total[label] += 1

                # average test loss
                test_loss = test_loss/len(test_loader.dataset)
                print('Test Loss: {:.6f}'.format(test_loss))

                for i in range(2):
                    if class_total[i] > 0:
                        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                            classes[i], 100 * class_correct[i] / class_total[i],
                            np.sum(class_correct[i]), np.sum(class_total[i])))
                    else:
                        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

                print('Test Accuracy (Overall): %2d%% (%2d/%2d)\n' % (
                    100. * np.sum(class_correct) / np.sum(class_total),
                    np.sum(class_correct), np.sum(class_total)))

For train category car:
For test gan biggan:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 2.145673
Test Accuracy of  Real: 62% (1249/2000)
Test Accuracy of  Fake: 50% (1011/2000)
Test Accuracy (Overall): 56% (2260/4000)

For test gan cyclegan:
For test category apple:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 1.403761
Test Accuracy of  Real: 56% (149/266)
Test Accuracy of  Fake: 43% (109/248)
Test Accuracy (Overall): 50% (258/514)

For test category horse:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 1.628230
Test Accuracy of  Real: 60% (73/120)
Test Accuracy of  Fake: 71% (100/140)
Test Accuracy (Overall): 66% (173/260)

For test category orange:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 1.025518
Test Accuracy of  Real: 64% (161/248)
Test Accuracy of  Fake: 54% (145/266)
Test Accuracy (Overall): 59% (306/514)

For test category summer:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 1.920667
Test A